# Demonstration of API to get flux limits

## Extracting a sensitivity cube from the HDF5

After installing the HETDEX API we can import the tools to read the
HDF5 sensivity cube containers

In [8]:
import tables

ImportError: /home/04120/dfarrow/.local/lib/python2.7/site-packages/tables/utilsextension.so: undefined symbol: PyUnicodeUCS4_Decode

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from hetdex_api.flux_limits.hdf5_sensitivity_cubes import SensitivityCubeHDF5Container

hdfcont = SensitivityCubeHDF5Container(filename="20181203_v013_sensitivity_cubes.h5")


ImportError: No module named tables

A method is supplied that shows the contents of the HDF5 container. This just calls the `print` function on the underlying `tables` File object. From this you can see what IFUs and shots are stored.

In [ ]:
hdfcont.list_contents()

We can extract a particular IFU like this. Note that you have to add the string 'ifuslot\_' to the three digit IFU slot. The `datevshot` is optional when only one shot flux limit is stored in a file (the default). Note that you have to append 'virus\_' to the datevshot. 

In [ ]:
scube = hdfcont.extract_ifu_sensitivity_cube("ifuslot_036", datevshot="virus_20181203v013")

## Using the sensitivity cube

Now we have the `SensitivityCube` object we can extract a flux limit, in erg/s/cm^2 from it like this

In [ ]:
# RA DEC (in degrees) Wavelength (Angstrom)
scube.get_f50(20.7384244, 0.4019458, 4240)

Locations out of range return 999, e.g.

In [ ]:
scube.get_f50(120.7384244, 13.4019458, 42400)

Of course we can also pass an arrays of coordinates, to get multiple results, e.g. to plot the flux limit versus wavelength

In [ ]:
from numpy import linspace
wls = linspace(3495.0, 5505.0, 100)
flims = scube.get_f50(20.7384244, 0.4019458, wls)
plt.plot(wls[flims < 999], flims[flims < 999]*1e16)
plt.xlabel("Wavelength (A)", fontsize=15.0)
plt.ylabel("Flux limit ($10^{-16}$ erg/s/cm$^2$)", fontsize=15.0)

To get the fraction of sources detected at a particular flux, rather than just the flux limit value one can use this method

In [ ]:
# RA DEC (in degrees) Wavelength (Angstrom) Flux (cgs)
scube.return_completeness(1.1e-17, 20.7384244, 0.4019458, 4245)

This completeness is computed using a functional form, this function takes two parameters the flux limit and a slope which controls how quickly completeness falls off. The latter is fixed to our best guess in this version of the API (**this means it might not be all that accurate!**), in the future this will all be better tested. By definition the flux limit stored in the cubes corresponds to 50% completeness, i.e.

In [ ]:
flux = scube.get_f50(20.7384244, 0.4019458, 4245)
scube.return_completeness(flux, 20.7384244, 0.4019458, 4245)

To finish off, here is a plot of the completeness versus flux at some position. Note we use the flux limit computed above to define the range of the plot

In [ ]:
fluxes = linspace(0.5*flux, 3.0*flux, 100)
compl = scube.return_completeness(fluxes, 20.7384244, 0.4019458, 4245)
plt.plot(fluxes*1e16, compl, "k-")
plt.xlabel("Flux ($10^{-16}$ erg/s/cm$^2$)")
plt.ylabel("Detected Fraction")

## Looping over all the IFUs in an HDF5 file
If you want to loop over all the IFUs then an iterator is provided. It returns pairs of IFU slot and sensitivity cube. In this example we overplot flux limits for all of the IFUs in the shot

In [2]:
for ifu_name, scube in hdfcont.itercubes():
    
    # Compute the central RA, DEC from the cubes WCS
    shape = scube.f50vals.shape
    ra, dec, lambda_ = scube.wcs.all_pix2world(shape[2]/2., shape[1]/2., shape[0]/2., 0)
    
    # Get the flux limits at this RA/DEC and a range of wavelengths
    flims = scube.get_f50(ra, dec, wls)

    # Plot
    plt.plot(wls[flims < 999], flims[flims < 999]*1e16, label=ifu_name)

plt.xlabel("Wavelength (A)", fontsize=15.0)
plt.ylabel("Flux limit ($10^{-16}$ erg/s/cm$^2$)", fontsize=15.0)
plt.show()

    

NameError: name 'hdfcont' is not defined